In [2]:
#ライブラリ
from crabnet.crabnet_ import get_data
from crabnet.data.materials_data import elasticity
from crabnet.crabnet_ import CrabNet
from os.path import join
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from crabnet.utils.data import get_data
from crabnet.crabnet_ import CrabNet
import vickers_hardness.data as vh_data
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import (
    GroupKFold,
    GroupShuffleSplit,
    KFold,
    ShuffleSplit,
)
from vickers_hardness.utils.plotting import parity_with_err
from vickers_hardness.vickers_hardness_ import VickersHardness

/workspaces/mat2vec_codespace/.venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X = pd.read_csv("/workspaces/mat2vec_codespace/main_code/final_SC_X_data.csv")
y = X["Tc"]
X = X.drop(columns=["Tc"])

In [4]:
display(X)

,formula
0,Hg0.66Pb0.34Ba2Ca1.98Cu2.9O8.4
1,Hg0.75Ba2.07Ca2.07Cu3.11O8.208
2,Hg0.75Ba2.07Ca2.07Cu3.11O8.187
3,Hg1Ba2Ca2Cu3O8.27
4,Hg1Ba2Ca2Cu3O8.29
...,...
18396,La3.5Ba3.5Cu7O16.415
18397,Nd1.85Ce0.15Cu1O4.045
18398,Y1Ba2Cu3O6.1
18399,Y0.97Ca0.03Ba2Cu3O6


In [5]:
# %% K-fold cross-validation
ss = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
cv = GroupKFold()
cvtype = "gcv"

groups = X["formula"]


trainval_idx, test_idx = list(ss.split(X, y, groups=groups))[0]

In [6]:
X_test, y_test = X.iloc[test_idx, :], y[test_idx]
X, y = X.iloc[trainval_idx, :], y.iloc[trainval_idx]

subgroups = X["formula"]

crabnet_dfs = []

In [9]:
for train_index, test_index in cv.split(X, y, subgroups):#上記のcvをしたそれぞれのデータセットのペアごとに処理を実行
    X_train, X_val = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    train_df = pd.DataFrame(
        {"formula": X_train["formula"], "target": y_train}
    )
    val_df = pd.DataFrame(
        {"formula": X_val["formula"], "target": y_val}
    )
    
    display(train_df)
    display(val_df)
    
    cb = CrabNet(#ここでCrabNetのモデルを作成 学習時に表示されている学習曲線の表示や学習時のbatch数などについてはここで設定を行う
                #今回のモデルだとデフォルトのbatch数は128
    verbose=True,
    random_state = 42,
    )
    
    break

,formula,target
0,Hg0.66Pb0.34Ba2Ca1.98Cu2.9O8.4,143.0
1,Hg0.75Ba2.07Ca2.07Cu3.11O8.208,135.8
2,Hg0.75Ba2.07Ca2.07Cu3.11O8.187,135.4
3,Hg1Ba2Ca2Cu3O8.27,135.0
7,Hg0.75Ba2.07Ca2.07Cu3.11O8.16,134.2
...,...,...
18393,Y0.88Ca0.12Ba2Cu3O6,0.0
18395,Ca0.75La0.25Fe1As2,0.0
18396,La3.5Ba3.5Cu7O16.415,0.0
18398,Y1Ba2Cu3O6.1,0.0


,formula,target
4,Hg1Ba2Ca2Cu3O8.29,135.0
5,Hg1Ba2Ca2Cu3O8.29,135.0
6,Hg0.75Ba2.07Ca2.07Cu3.11O8.15,134.7
11,Hg0.6Ti0.4Ba2Ca2Cu3O8.4,134.0
21,Hg0.85Re0.15Ba1.66Sr0.34Ca2Cu3O8,131.3
...,...,...
18366,Ca0.4M0.41Mo1S2,0.0
18368,Y1In2.73Sn0.27,0.0
18378,Ca4Al2Fe2As0.1P1.9O6,0.0
18394,Ca0.81Ce0.19Fe1As2,0.0



Model architecture: out_dims, d_model, N, heads
3, 512, 3, 4
Running on compute device: cpu


In [10]:

cb.fit(train_df)#モデル訓練　内容は不明だがもしかして完成したモデルの出力も行われている可能性がある

y_pred, y_std, y_true,  = cb.predict(test_df = val_df, return_uncertainty = True, return_true = True)#原文ママではエラー　原因は出力が4つも出てこないということ、実際オプションでy_trueおよびy_stdを追加することは可能
#ただし、何もオプションを入れていない時点では、予測された結果値のみが出力される
#得られるstdは予測された値の標準偏差を示している
crabnet_dfs.append(#答えのファイルの作成
    pd.DataFrame(
        {
            "actual_Tc": y_true,
            "predicted_Tc": y_pred,
            "y_std": y_std,
            "formula": val_df["formula"],
        }
    )
)

Model size: 11987206 parameters



Generating EDM: 100%|██████████| 10648/10648 [00:00<00:00, 139107.71formulae/s]
invalid value encountered in true_divide


skipping composition OrderedDict([('Se', 2.0), ('Ca', 1.0), ('M', 1.0), ('Mo', 1.0)])
skipping composition OrderedDict([('S', 2.0), ('M', 1.0), ('Ta', 1.0)])
skipping composition OrderedDict([('M', 1.0), ('Se', 1.0), ('Fe', 0.995), ('Li', 0.6), ('V', 0.01)])
skipping composition OrderedDict([('Se', 2.0), ('M', 1.0), ('Ti', 1.0), ('Li', 0.1)])
skipping composition OrderedDict([('Se', 2.0), ('M', 1.0), ('Ti', 1.0), ('Li', 0.08)])
skipping composition OrderedDict([('Se', 2.0), ('Fe', 1.86), ('Na', 0.8), ('M', 0.6)])
skipping composition OrderedDict([('V', 2.0), ('Hf', 1.0), ('D', 0.5)])
skipping composition OrderedDict([('Se', 2.0), ('M', 1.0), ('Ti', 1.0), ('Li', 0.2)])
skipping composition OrderedDict([('S', 2.0), ('Mo', 1.0), ('M', 0.4), ('Sr', 0.17)])
skipping composition OrderedDict([('Zr', 1.0), ('N', 1.0), ('Cl', 1.0), ('Li', 0.15), ('M', 0.08)])
skipping composition OrderedDict([('S', 2.0), ('M', 1.0), ('Ta', 1.0)])
skipping composition OrderedDict([('S', 2.0), ('M', 1.0), ('Ta', 

Generating EDM: 100%|██████████| 2662/2662 [00:00<00:00, 147079.38formulae/s]


skipping composition OrderedDict([('M', 1.0), ('Hf', 1.0), ('N', 1.0), ('Cl', 1.0), ('Ca', 0.24)])
skipping composition OrderedDict([('Fe', 2.05), ('Se', 2.0), ('Li', 1.25), ('M', 0.64)])
skipping composition OrderedDict([('M', 1.0), ('Hf', 1.0), ('N', 1.0), ('Cl', 1.0), ('Ca', 0.11)])
skipping composition OrderedDict([('S', 2.0), ('M', 1.0), ('Ta', 1.0)])
skipping composition OrderedDict([('S', 2.0), ('M', 1.0), ('Ta', 1.0)])
skipping composition OrderedDict([('S', 2.0), ('M', 1.0), ('Ta', 1.0)])
skipping composition OrderedDict([('S', 2.0), ('Ta', 1.0), ('M', 0.5)])
skipping composition OrderedDict([('Hf', 1.0), ('N', 1.0), ('Cl', 1.0), ('Li', 0.5), ('M', 0.3)])
skipping composition OrderedDict([('S', 2.0), ('M', 1.0), ('Ta', 1.0)])
skipping composition OrderedDict([('Os', 2.0), ('Ls', 0.91), ('Gd', 0.09)])
skipping composition OrderedDict([('M', 1.0), ('Hf', 1.0), ('N', 1.0), ('Cl', 1.0), ('Ba', 0.1)])
skipping composition OrderedDict([('Se', 2.0), ('Pf', 1.0), ('Nb', 1.0)])
skippin

IndexError: index 9223372036854775807 is out of bounds for dimension 0 with size 5000